# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
#max_steps = 5e5 # Set maximum number of steps to run environment.
max_steps = 3400e4
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "wizim" # Name of the training environment file.
curriculum_file = "E:\work\ml-agents\python\curricula\wizim.json" #None #

#"E:\work\ml-agents\python\curricula\wizim.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-5 # Model learning rate.
hidden_units = 32 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'WizimAcademy' started successfully!


Unity Academy name: WizimAcademy
        Number of brains: 1
        Reset Parameters :
		scale -> 1.0
Unity brain name: Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 6
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
        
    print(max_steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/ppo\model-6750000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-6750000.cptk


34000000.0
Saved Model


INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	scale -> 0.8
INFO:unityagents:
Lesson changed. Now in Lesson 2 : 	scale -> 0.85
INFO:unityagents:
Lesson changed. Now in Lesson 3 : 	scale -> 0.87


Step: 6751000. Mean Reward: 0.8713074380165291. Std of Reward: 0.3300969875420636.


...

Step: 32109000. Mean Reward: 0.5302828467153285. Std of Reward: 0.5930549160924183.
Step: 32110000. Mean Reward: 0.5430751341681574. Std of Reward: 0.5854273286630497.
Step: 32111000. Mean Reward: 0.578414364640884. Std of Reward: 0.5720963449748829.
Step: 32112000. Mean Reward: 0.5815652951699463. Std of Reward: 0.5700536919863551.
Step: 32113000. Mean Reward: 0.5203406593406593. Std of Reward: 0.6032996640839562.
Step: 32114000. Mean Reward: 0.5519796296296295. Std of Reward: 0.58450391560949.
Step: 32115000. Mean Reward: 0.5888025594149908. Std of Reward: 0.5699573457795545.
Step: 32116000. Mean Reward: 0.587090573012939. Std of Reward: 0.5721045019160491.
Step: 32117000. Mean Reward: 0.5360608856088561. Std of Reward: 0.5933710950497539.
Step: 32118000. Mean Reward: 0.5644014466546112. Std of Reward: 0.5779819053945799.
Step: 32119000. Mean Reward: 0.5532787769784172. Std of Reward: 0.5889543987391449.
Step: 32120000. Mean Reward: 0.538616513761468. Std of Reward: 0.588237680308423

Step: 32207000. Mean Reward: 0.5065394736842105. Std of Reward: 0.5951963790589084.
Step: 32208000. Mean Reward: 0.5528218181818182. Std of Reward: 0.5871121381786397.
Step: 32209000. Mean Reward: 0.5821567567567567. Std of Reward: 0.5773504211141207.
Step: 32210000. Mean Reward: 0.5669524680073126. Std of Reward: 0.582528091709765.
Step: 32211000. Mean Reward: 0.5647546816479401. Std of Reward: 0.5838565778419544.
Step: 32212000. Mean Reward: 0.5242801484230055. Std of Reward: 0.5851853213234003.
Step: 32213000. Mean Reward: 0.5384101123595505. Std of Reward: 0.58967613656995.
Step: 32214000. Mean Reward: 0.5092950819672132. Std of Reward: 0.5942841497394581.
Step: 32215000. Mean Reward: 0.5690614525139666. Std of Reward: 0.5825893350219489.
Step: 32216000. Mean Reward: 0.5439247311827957. Std of Reward: 0.5917918705005326.
Step: 32217000. Mean Reward: 0.5884803001876173. Std of Reward: 0.5714132426620782.
Step: 32218000. Mean Reward: 0.5550455373406192. Std of Reward: 0.5969062923004

Step: 32305000. Mean Reward: 0.5766924528301887. Std of Reward: 0.5802880853666668.
Step: 32306000. Mean Reward: 0.5354611307420495. Std of Reward: 0.5954752632353616.
Step: 32307000. Mean Reward: 0.5717481884057971. Std of Reward: 0.5757255032297787.
Step: 32308000. Mean Reward: 0.5794401408450705. Std of Reward: 0.5868046588810344.
Step: 32309000. Mean Reward: 0.5783393177737881. Std of Reward: 0.5840128943833327.
Step: 32310000. Mean Reward: 0.5697236842105263. Std of Reward: 0.579981767249035.
Step: 32311000. Mean Reward: 0.5579259927797834. Std of Reward: 0.5756576775054302.
Step: 32312000. Mean Reward: 0.4891336996336996. Std of Reward: 0.6141554271662952.
Step: 32313000. Mean Reward: 0.545051376146789. Std of Reward: 0.5874473133689053.
Step: 32314000. Mean Reward: 0.572111111111111. Std of Reward: 0.5840976916042222.
Step: 32315000. Mean Reward: 0.600386404293381. Std of Reward: 0.5694570406753502.
Step: 32316000. Mean Reward: 0.5844583333333333. Std of Reward: 0.56888340136696

Step: 32403000. Mean Reward: 0.5878180212014134. Std of Reward: 0.580328507942757.
Step: 32404000. Mean Reward: 0.5833339285714285. Std of Reward: 0.5864426542348321.
Step: 32405000. Mean Reward: 0.5560648648648648. Std of Reward: 0.5822706526261361.
Step: 32406000. Mean Reward: 0.579032667876588. Std of Reward: 0.5759787904002074.
Step: 32407000. Mean Reward: 0.5306925925925926. Std of Reward: 0.5969712645394845.
Step: 32408000. Mean Reward: 0.6146556776556776. Std of Reward: 0.5614359260187584.
Step: 32409000. Mean Reward: 0.5505921787709497. Std of Reward: 0.5844832056329314.
Step: 32410000. Mean Reward: 0.528166037735849. Std of Reward: 0.5843928689346397.
Step: 32411000. Mean Reward: 0.5571062618595826. Std of Reward: 0.5767409154768521.
Step: 32412000. Mean Reward: 0.574325. Std of Reward: 0.5830174416191534.
Step: 32413000. Mean Reward: 0.5588454376163874. Std of Reward: 0.5782830333091772.
Step: 32414000. Mean Reward: 0.5446963946869069. Std of Reward: 0.5888817811972883.
Step:

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-32400000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-32400000.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
